# Import packages

In [ ]:
import os
import re
import pandas as pd
import sklearn
import nltk
import unicodedata
from dotenv import load_dotenv

# Load data

In [ ]:
# Download nltk data packages
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)

In [ ]:
# Load the environment variables
load_dotenv()

# Get the paths to the data
data_dir = os.getenv("DATA_DIR")
articles_dir = os.path.join(data_dir, "articles")

# Load the main dataset
df = pd.read_csv(f"{data_dir}/article_info_V2.csv", index_col=0, parse_dates=[1])

# Load the external dataset
external_df = pd.read_csv(f"{data_dir}/other-articles.csv", index_col=0, parse_dates=[1], sep=";")

# Clear the data directory variable
del data_dir

# Cleaning

In [ ]:
# Drop unused columns
df.drop(columns=["Author", "Type", "Keywords"], inplace=True)

# Drop rows with empty date values
df.dropna(subset=["Date"], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Remove the url column from the external dataset
external_df.drop(columns=["Url"], inplace=True)

# Merging datasets

In [ ]:
# Add the external dataset to the main dataset
df = pd.concat([df, external_df])

# Reset the index
df.reset_index(drop=True, inplace=True)

# Clear the external dataset variable
del external_df

# Remove duplicates

In [ ]:
# Count the number of entries before duplicate removal
entries_count = len(df)

# remove duplicate entries by title and date
df.drop_duplicates(subset=["Title", "Date"], keep="first", inplace=True)

# Print the number of deleted entries
print(f"Removed {entries_count - len(df)} duplicate entries.")

# Clear the entries count variable
del entries_count

# Parse target

In [ ]:
# Define functions to parse target
def clean_string(str):
    # Make the string lowercase
    str = str.lower()
    # Remove all non-alphanumeric characters
    str = re.sub(r"[^\w\s]", "", str)
    # Remove the leading and trailing spaces
    return str.strip()

def parse_tags(str):
    # Split the string on each comma
    raw_list = str.split(",")
    # Clean every string in the list
    return list(map(clean_string, raw_list))

In [ ]:
# Convert the raw string values of the Tags column to lists of strings
df["Tags"] = df["Tags"].apply(parse_tags)

In [ ]:
# Clear the cleaning and parsing functions
del clean_string, parse_tags

# Load articles

In [ ]:
# Article loading function
def load_article(title):
    # Create filename from title
    file_name = f"{title}.txt"
    # Get the path of the article
    file_path = os.path.join(articles_dir, file_name)
    try:
        # Attempt to read the article
        file = open(file_path, "r", encoding="utf-8")
    except:
        # If the article could not be read, return None
        return None
    # Read the contents of the article
    contents = file.read()
    # Close the file
    file.close()
    # Return the contents of the article
    return contents

In [ ]:
# Load the articles
df["Contents"] = df["Title"].apply(load_article)

# Clear the article loading function
del load_article

In [ ]:
# Count the number of entries in the dataframe
entries_count = len(df)

# Remove rows without article contents
df.dropna(subset=["Contents"], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Print the number of deleted entries
print(f"Removed {entries_count - len(df)} entries without article contents.")

# Clear the entries count variable
del entries_count

# Article cleaning

In [ ]:
# Remove all newline characters
def remove_newlines(text):
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    return text

# Replace shortened grammar with full grammar
def replace_grammar(text):
    text = text.replace("it's", "it is")
    text = text.replace("he's", "he is")
    text = text.replace("she's", "she is")
    text = text.replace("what's", "what is")
    text = text.replace("n't", "not")
    text = text.replace("'scuse", " excuse")
    text = text.replace("'s", "")
    text = text.replace("'t", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ll", " will")
    text = text.replace("'ve", " have")
    text = text.replace("'d", " would")
    text = text.replace("'m", " am")
    return text

# Replace accented characters (like é, ë, ï) with their unaccented counterparts (e, e, i)
def remove_accented_characters(text):
    return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

# Remove punctuation from text (like commas, periods, and exclamation marks)
def remove_punctuation(text):
    return re.sub(r"[^\w\s]", "", text)

# Remove digits from text
def remove_digits(text):
    return re.sub(r"\d+", "", text)

# Remove extra whitespace characters
def remove_extra_whitespace(text):
    # Remove spaces at the start and end of the text
    text = text.strip()
    # Remove double whitespace characters
    return re.sub(r"\s+", " ", text)

# Convert uppercase text to lowercase text
def lowercase(text):
    return text.lower()

# Testing below

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_train = pd.DataFrame(df["Contents"][0:10])
X_train.rename(columns={'Contents': 'speech'}, inplace=True)
X_train

In [ ]:
def preprocess_text(text):
    # Remove newlines
    text = remove_newlines(text)
    # Replace accented characters (like é, ë, ï) with their unaccented counterparts (e, e, i)
    text = remove_accented_characters(text)
    # Replace shortened grammar with full grammar
    text = replace_grammar(text)
    # Remove punctuation from text (like commas, periods, and exclamation marks)
    text = remove_punctuation(text)
    # Remove digits
    text = remove_digits(text)
    # Remove extra whitespace characters
    text = remove_extra_whitespace(text)
    # Convert uppercase text to lowercase text
    text = lowercase(text)

    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos="v") for token in tokens]

    # Remove stopwords
    keywords = [lemma for lemma in lemmas if lemma not in stopwords.words("english")]

    return keywords

In [ ]:
# Import module
from sklearn.feature_extraction.text import TfidfVectorizer
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to tf-idf
X_train = vectoriser.fit_transform(X_train["speech"])

In [ ]:
pd.DataFrame(X_train.toarray(), columns=vectoriser.get_feature_names_out())